## Download Files

In [ ]:
!rm -rf HRTFCNN
!rm -rf data/cipic_hrtf_sofa
!rm ear.zip
!rm pinnas.zip
!rm -rf binural-updates
!rm -rf CIPIC_hrtf_database

In [ ]:
!pip install h5py
!pip install sofar
!pip install packaging

In [ ]:
%cd /content/

In [ ]:
!git clone https://github.com/docEdub/HRTFCNN.git

In [ ]:
%cd HRTFCNN

In [ ]:
!git pull origin master

In [ ]:
!ls

In [ ]:
!wget -r -l1 -w 1 --no-parent http://sofacoustics.org/data/database/cipic/

In [ ]:
!mkdir data/cipic_hrtf_sofa
!mv sofacoustics.org/data/database/cipic/*.sofa data/cipic_hrtf_sofa
!ls data/cipic_hrtf_sofa | wc -l

In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

                          
# 3. Load a file by ID and print its contents.
downloaded = drive.CreateFile({'id': '1a6LihoO2agENYDM6qdpL9tqUa4yQBHTn'})
print('Downloaded pinnas "{}"'.format(downloaded.GetContentFile('pinnas.zip')))

downloaded = drive.CreateFile({'id': '1rJT5NgX_OoI_5_fFxQmjGvxo6ZgFSvN_'})
print('Downloaded images "{}"'.format(downloaded.GetContentFile('ear.zip')))

In [ ]:
!unzip pinnas.zip
!unzip ear.zip
!mv CIPIC_hrtf_database data
!mv binural-updates/ear_photos data

In [ ]:
!ls utils

##Load Data

In [ ]:
import os
import sys
import scipy.io
import scipy.signal
import logging
import numpy as np
import matplotlib.pyplot as plt
import cv2
from utils.hrtf import *
from utils.image_utils import *

In [ ]:
hrtf = CipicHRTF('data/cipic_hrtf_sofa/subject_003.sofa', 44100.0)

In [ ]:
print(hrtf.azimuths)

In [ ]:
# get pinnas
pinnas = np.array(scipy.io.loadmat('./data/CIPIC_hrtf_database/anthropometry/anthro.mat')['X'])

In [ ]:
def get_image(folder, num):
    num_str = str(num)
    if num < 100:
        num_str = '0' + num_str
    if num < 10:
        num_str = '0' + num_str
    num = num_str
    try:
        return plt.imread(folder+ 'Subject_'+str(num)+'/'+str(num)+'_left_side.jpg'), 0
    except:
        try:
            return plt.imread(folder+ 'Subject_'+str(num)+'/'+str(num)+'_right_side.jpg'), 1
        except:
            try:
                return plt.imread(folder+ 'Subject_'+str(num)+'/0'+str(num)+'_left.jpg'), 0
            except:
                try:
                    return plt.imread(folder+ 'Subject_'+str(num)+'/0'+str(num)+'_right.jpg'), 1
                except:
                    try:
                        return plt.imread(folder+ 'Subject_'+str(num)+'/0'+str(num)+'_left.JPG'), 0
                    except:
                        try:
                            return plt.imread(folder+ 'Subject_'+str(num)+'/0'+str(num)+'_right.JPG'), 1
                        except:
                            try:
                                return plt.imread(folder+ 'Subject_'+str(num)+'/Subject_'+str(num)+'_left_side.jpg'), 0
                            except:
                                try:
                                    return plt.imread(folder+ 'Subject_'+str(num)+'/0'+str(num)+'_left_2.jpg'), 0
                                except:
                                    return plt.imread(folder+ 'Subject_'+str(num)+'/00'+str(58)+'_left.jpg'), 0

In [ ]:
valid_ranges = []
valid_images = []
valid_final = []
valid_image_ranges = []
for i in range(200):
    try:
        get_hrtf_sofa('data/cipic_hrtf_sofa/', i)
        valid_ranges.append(i)
        try:
            img, ear_num = get_image('data/ear_photos/', i)
            valid_images.append(img)
            valid_image_ranges.append((i, ear_num))
        except FileNotFoundError as err:
            print(err)
            print('Image not Found', i)
            valid_images.append(None)
            valid_image_ranges.append(None)
    except:
        pass
#         print('Failed to load sofa', i)
valid_image_ranges = np.array(valid_image_ranges)

In [ ]:
pinna_vals = [pinnas[i] for i, val in enumerate(valid_ranges)]
print(len(valid_ranges))
print(len(pinna_vals))
print(len(valid_images))
print(len(valid_image_ranges))

In [ ]:
print(valid_image_ranges)

In [ ]:
valid_final = np.array(valid_ranges)[~np.isnan(pinna_vals).any(axis=1)]
valid_images = np.array(valid_images)[~np.isnan(pinna_vals).any(axis=1)]
valid_image_ranges = np.array(valid_image_ranges)[~np.isnan(pinna_vals).any(axis=1)]
pinna_vals = np.array(pinna_vals)[~np.isnan(pinna_vals).any(axis=1)]

In [ ]:
print(len(valid_final))
print(len(valid_images))
print(len(valid_image_ranges))
print(len(pinna_vals))

In [ ]:
print(valid_final)
print(valid_image_ranges)

In [ ]:
valid_images = [valid_images[idx] for idx, val in enumerate(valid_image_ranges) if val is not None]
pinna_vals = [pinna_vals[idx] for idx, val in enumerate(valid_image_ranges) if val is not None]
valid_image_ranges = [valid_image_ranges[idx] for idx, val in enumerate(valid_image_ranges) if val is not None]

In [ ]:
print(len(valid_final))
print(len(valid_images))
print(len(valid_image_ranges))
print(len(pinna_vals))

In [ ]:
impulse_vals = []
for i, ear in valid_image_ranges:
    impulse_vals.append(get_hrtf_sofa('data/cipic_hrtf_sofa/', i).impulses[:,ear,:])




In [ ]:
print(impulse_vals)

In [ ]:
print(np.array(impulse_vals).shape)

In [ ]:
valid_images_canny = []
for i in valid_images:
    valid_images_canny.append(np.expand_dims(get_canny_image(i), axis=2))
    

In [ ]:
elevation_vals = get_hrtf_sofa('data/cipic_hrtf_sofa/', 3).elevations
azimuths_vals = get_hrtf_sofa('data/cipic_hrtf_sofa/', 3).azimuths

In [ ]:
print(elevation_vals.shape)

In [ ]:
hrtf_dataset = []
images_dataset = []
hrtf_Y = []

for idx, person in enumerate(np.array(impulse_vals)):
    for jdx, hrtf_data in enumerate(person):
        hrtf_dataset.append(np.hstack((elevation_vals[jdx], azimuths_vals[jdx], pinna_vals[idx])))
        images_dataset.append(valid_images_canny[idx])
        hrtf_Y.append(hrtf_data)

In [ ]:
hrtf_dataset = np.array(hrtf_dataset)
hrtf_Y = np.array(hrtf_Y)
images_dataset = np.array(images_dataset)

## Train Model

In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import tensorflow as tf
import time
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.layers import Input, Embedding, LSTM, Dense, Conv2D,MaxPooling2D, Flatten, concatenate
# from keras.models import Model
import tensorflow as tf
from keras import backend as K
# from tensorflow.keras import Model

def root_mean_squared_logarithmic_error(y_true, y_pred):
    return 20*K.log(K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))) / K.log(10.0)
    
def get_model():
    main_input = tf.keras.layers.Input(shape=(19,), name='main_input')

    x = tf.keras.layers.Dense(64, activation='relu')(main_input)
    x = tf.keras.layers.Dense(64, activation='relu')(x) 
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    anthro_out = tf.keras.layers.Dense(8, activation='relu')(x)
   
    photo_input = tf.keras.layers.Input(shape=(64,64,1), name='photo_input')
    
    y = tf.keras.layers.Conv2D(16, (3, 3), activation='relu')(photo_input)
    y = tf.keras.layers.MaxPooling2D((2, 2))(y)
    y = tf.keras.layers.Conv2D(16, (3, 3), activation='relu')(y)
    y = tf.keras.layers.MaxPooling2D((2, 2))(y)
    y = tf.keras.layers.Flatten()(y)
    photo_output = tf.keras.layers.Dense(8, activation='relu')(y)

    z = tf.keras.layers.concatenate([anthro_out, photo_output])
    
    z = tf.keras.layers.Dense(64, activation='relu')(z) 
    z = tf.keras.layers.Dense(64, activation='relu')(z) 
    z = tf.keras.layers.Dense(64, activation='relu')(z) 
    output = tf.keras.layers.Dense(200, activation='relu')(z) 

    return tf.keras.Model(inputs=[main_input, photo_input], outputs=[output])

X_train, X_test, y_train, y_test, image_train, image_test = train_test_split(
    hrtf_dataset, hrtf_Y, images_dataset, test_size=0.33, random_state=42)

dnn_model = get_model()

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_host(resolver.master())
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

dnn_model.compile(
    optimizer=tf.keras.optimizers.Adagrad(learning_rate=1e-3),
    loss=root_mean_squared_logarithmic_error,
    metrics=[root_mean_squared_logarithmic_error, 'mean_squared_logarithmic_error', 'mse'])

dnn_model.summary()


In [ ]:
use_tpu = True #@param {type:"boolean"}

if use_tpu:
    assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

if 'COLAB_TPU_ADDR' in os.environ:
  TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
else:
  TF_MASTER=''

with tf.compat.v1.Session(TF_MASTER) as session:
  print ('List of devices:')
  print(session.list_devices())

In [ ]:
print(X_train.shape, image_train.shape, y_train.shape)

In [ ]:
history = dnn_model.fit(
  [X_train,image_train], y_train,
  validation_split = 0.2,
  steps_per_epoch = 100,
  validation_steps=50,
  epochs=5,
  shuffle=True
)

In [ ]:
plt.plot(history.history['mse'])

In [ ]:
plt.plot(history.history['root_mean_squared_logarithmic_error'])

In [ ]:
dnn_model.evaluate([X_test,image_test], y_test,
    steps = 50, verbose=1)


In [ ]:
dnn_model.save_weights('./hrtf_tpu.h5', overwrite=True)

## Predict

In [ ]:
!ls

In [ ]:
#Enter 17 Measurements Here
anthro = np.array([
    11.310303, # 01) head width
    24.104653, # 02) head height
    24.290701, # 03) head depth
    7.894130,  # 04) pinna offset down
    7.961877,  # 05) pinna offset back
    7.879897,  # 06) neck width
    8.825650,  # 07) neck height
    13.574393, # 08) neck depth
    26.341024, # 09) torso top width
    13.106498, # 10) torso top height
    22.631457, # 11) torso top depth
    25.762224, # 12) shoulder width
    6.325874,  # 13) head offset forward
    170.780,   # 14) height
    127.000,   # 15) seated height
    55.88,     # 16) head circumference
    40.64      # 17) shoulder circumference
])

canny_image = np.array([np.expand_dims(get_canny_image(plt.imread('ear_image.jpg')), axis=2)])

In [ ]:
dnn_model.load_weights('hrtf_tpu.h5')
final_impulses = []
for i in range(len(hrtf.elevations)):
    # Keras docs: https://keras.io/api/models/model_training_apis/#predict-method
    # Keras layers explanation: https://wandb.ai/ayush-thakur/dl-question-bank/reports/Keras-Layer-Input-Explanation-With-Code-Samples--VmlldzoyMDIzMDU
    main_input = np.hstack((anthro, hrtf.elevations[i], hrtf.azimuths[i])) # makes main_input be in shape (19,).
    main_input = np.expand_dims(main_input, axis=0) # main_input needs to be in shape (1, 19), so add a dimension before passing it to `predict`.
    impulse = dnn_model.predict_on_batch([main_input, canny_image])[0]
    final_impulses.append([impulse, impulse]) # assumes left ear is same as right ear

In [ ]:
elevations, azimuths = interauralPolarToVerticalPolarCoordinates(hrtf.elevations, hrtf.azimuths)
create_cipic_hrtf('data/template.sofa', 'data/predict.sofa', final_impulses, hrtf.elevations, hrtf.azimuths)

In [ ]:
from google.colab import files
files.download('data/predict.sofa')